This document illustrates how to use all functionality 
of 'methylseqextractor' 
[https://github.com/perishky/methylseqextractor](https://github.com/perishky/methylseqextractor).

## Import methylseqextractor modules

All necessary modules can be imported as follows:


In [ ]:
#| eval: false
from methylseqextractor import (
  Window,
  MethylSeqDataset,
  LevelCalculator,
  ClonalFlipCounter,
  ConcurrenceCalculator,
  CAMDACalculator
)

In [ ]:
#| echo: false
from src import (
  Window,
  MethylSeqDataset,
  LevelCalculator,
  ClonalFlipCounter,
  ConcurrenceCalculator,
  CAMDACalculator
)

In the examples below, we'll use `pandas` to
store outputs as data frames.


In [ ]:
import pandas as pd

## Extract reads containing specified CpG sites

We can extract the reads overlapping each of a selected set of CpG sites. 

Here we prepare to extract reads overlapping CpG sites in the 
region chr1:1245000-1246000.


In [ ]:
bamfn = "data/sample.bam"
fastafn = "genome/hg19.fa"
dataset = MethylSeqDataset(bamfn, fastafn)
iter = dataset.methylation("chr1",1245000,1246000)

Each step of this iterator will return the reads of a different CpG site. 
Here are the reads for the first CpG site. 


In [ ]:
sites = next(iter)
pd.DataFrame([site.get_dict() for site in sites])

And the second CpG site.


In [ ]:
sites = next(iter)
pd.DataFrame([site.get_dict() for site in sites])

## Calculate DNA methylation levels

A variety of summary statistics can be calculated from these read sets. 

One is the proportion of reads in which a CpG site is methylated 

Here we calculate DNA methylation levels for each CpG site 
in the region chr1:1245000-1246000.


In [ ]:
levels = LevelCalculator(dataset)
iter = levels.calculate("chr1", 1245000, 1246000)
pd.DataFrame([site for site in iter])

CpG sequences are symmetric on DNA strands.
By default, `MethylSeqDataset` merges these as if they
are a single genomic position.
It is possible to prevent merging. 


In [ ]:
dataset_stranded = MethylSeqDataset(bamfn, fastafn, merge_strands=False)
levels = LevelCalculator(dataset_stranded)
iter = levels.calculate("chr1", 1245000, 1246000)
pd.DataFrame([site for site in iter])

## View DNA methylation patterns

These read sets can also be simplified to just 
retrieve the DNA methylation patterns within 
a window of specified size as it 'slides' across the genome.

Here we show the DNA methylation patterns
200 base-pairs at a time.


In [ ]:
window = Window(dataset,200)
iter = window.slide("chr1", 1245000, 1246000)
print(next(iter)) ## first window position
print(next(iter)) ## second window position

## Count clonal flips

From these DNA methylation patterns, we can calculate
how DNA methylation varies within and between reads. 
There are a variety of such metrics. 

One metric calculates the number of times the DNA methylation 
of CpG sites changes as one moves from one end of a 
DNA fragment (corresponding to a read pair) to the other. 


In [ ]:
flips = ClonalFlipCounter(dataset,200)
iter = flips.calculate("chr1", 1245000, 1246000)
pd.DataFrame([ region for region in iter])

## Calculate CAMDA scores 

The aim of **Concurrence of Active Methylation and De-methylAtion (CAMDA)**
scores is to identify genomic regions where DNA methylation-modifying 
enzymes are active.  These regions are assumed to be those 
with many partially methylated reads. 

The original implementation can be found here:
[https://github.com/JiejunShi/CAMDA](https://github.com/JiejunShi/CAMDA).

The approach is illustrated below with an example:

![CAMDA Schematic](https://raw.githubusercontent.com/JiejunShi/CAMDA/master/images/CAMDA_schematic.png)

Not shown in the figure is the fact that CAMDA scores can be calculated for individual CpG sites
(i.e. the proportion of times a CpG site is a concurrence site)
or for regions of a given size
(i.e. the proportion of times CpG sites in a region are concurrence sites).

The 'size' argument to `CAMDACalculator`
sets the size of the CpG-centered region for the CAMDA score.
In the original CAMDA package, this is not an option and is always
set to 1 (as below).


In [ ]:
camda = CAMDACalculator(dataset, 1)
iter = camda.calculate("chr1", 1245000, 1246000)
pd.DataFrame([site for site in iter])

## Calculate concurrence scores

We can similarly calculate DNA methylation concurrence scores, 
i.e. in a given window, 
the proportion of unmethylated CpG sites in a read with at least
one methylated CpG site relative to the set of all CpG sites 
in all reads.

**Note:** this is different from the published 'CAMDA' score
calculated above which performs these calculations across 
entire reads rather than just within the window. 
We restrict the calculation to the window
to avoid an overdependence of the calculated statistics on read length.


In [ ]:
concurrence = ConcurrenceCalculator(dataset,200)
iter = concurrence.calculate("chr1", 1245000, 1246000)
pd.DataFrame([region for region in iter])

## Analysing the entire genome efficiently using multiple processors

Outputs are likely to get large and computationally more costly
when generated for the entire genome rather than just a single region.
Fortunately, these analyses are naturally parallalizable
by chromosome using the `multiprocessing`library.

For example, here we count DNA methylation 'flip' statistics 
chromosome-by-chromosome and save the results to CSV files.


In [ ]:
#| eval: false

import multiprocessing
concurrence = ConcurrenceCalculator(dataset,200)

def calculate_chrom_concurrences(chrom):
    iter = concurrence.calculate(chrom)
    return pd.DataFrame([region for region in iter])

chromosomes = ["chr"+str(i) for i in range(1,23)] + ["chrX"]

with multiprocessing.Pool(processes=12) as pool:
    for stats in pool.imap(calculate_chrom_concurrences, chromosomes):
        if len(stats) > 0:
            stats.to_csv("concurrences_" + stats['chrom'][0] + ".csv")